In [1]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

import numpy as np
import os

print(tf.__version__)
print(tf.keras.__version__)

1.11.0
2.1.6-tf


In [7]:
# only the first card is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# display memory auto allocation
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
keras.backend.set_session(session)

## import data

In [8]:
%%time
x_val = np.load("data/x_val.npy")

x_val = keras.applications.vgg16.preprocess_input(x_val)

CPU times: user 15.5 s, sys: 45.1 s, total: 1min
Wall time: 1min 27s


In [9]:
y_val = np.load("data/y_val.npy")
y_val = y_val.astype(int)

#y_val_one_hot = keras.utils.to_categorical(y_val, 1000)

## test 

In [10]:
def top_k_accuracy(y_true, y_pred, top=1):
    '''From: https://github.com/chainer/chainer/issues/606
    
    Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = y_pred.argsort()[:, -top:]
    #argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true, axis=0).mean()

## resnet50

In [12]:
# vgg16 model
model = keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [13]:
y_pred = model.predict(x_val, batch_size=64, verbose=1)

50000/50000 [==============================] - 200s 4ms/step


### top-1 accuracy of ResNet50 in official document: 0.749

In [14]:
top_k_accuracy(y_val, y_pred, top=1)

0.74928

### top-5 accuracy of ResNet50 in official document: 0.921

In [15]:
top_k_accuracy(y_val, y_pred, top=5)

0.9206